In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
import re
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModel
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.io import read_image
from transformers import ViTModel

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
 
nltk.download('stopwords')

def clean_text(phrase) :
    
    stop_words = set(stopwords.words('english'))
    phrase = phrase.lower()
    phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)
    word_tokens = word_tokenize(phrase)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
     
    for w in word_tokens:
        if w not in stop_words:
            if set(w) != {'x'} :
                filtered_sentence.append(w)
    
    filtered_sentence = ' '.join(filtered_sentence)
    filtered_sentence = filtered_sentence.replace('year old', "")
    filtered_sentence = filtered_sentence.replace('nan', "")
    return filtered_sentence

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manasamangipudi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
reports

,uid,MeSH,Problems,image,indication,comparison,findings,impression,text
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size a...,Normal chest x-XXXX.,Positive TB test The cardiac silhouette and me...
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy X...,No acute pulmonary findings.,Preop bariatric surgery. Borderline cardiomega...
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,nan,"No displaced rib fractures, pneumothorax, or p...","rib pain after a XXXX, XXXX XXXX steps this XX..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and ...,1. Bullous emphysema and interstitial fibrosis...,XXXX-year-old XXXX with XXXX. There are diffus...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonar...,No acute cardiopulmonary abnormality.,Chest and nasal congestion. The cardiomediasti...
...,...,...,...,...,...,...,...,...,...
3846,3995,Lung/hyperdistention/mild;Diaphragm/bilateral/...,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,Xray Chest PA and Lateral,"Nausea, vomiting x2 weeks. Dialysis patient.","XXXX, XXXX.",The cardiomediastinal silhouette and pulmonar...,1. Interval resolution of bibasilar airspace d...,"Nausea, vomiting x2 weeks. Dialysis patient. T..."
3847,3996,Spine/degenerative,Spine,Xray Chest PA and Lateral,NaN,None.,The lungs are clear. Heart size is normal. No...,Clear lungs. No acute cardiopulmonary abnormal...,NaN
3848,3997,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodul...","No acute findings, no evidence for active TB.",XXXX-year-old male with positive PPD. Heart si...
3849,3998,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,nan,Heart size is normal and the lungs are clear.,tuberculosis positive PPD nan


In [5]:
reports = pd.read_csv('../archive/indiana_reports.csv')
image_files = pd.read_csv('../archive/indiana_projections.csv')

reports['findings'] = reports['findings'].apply(lambda x : x if x != None else '')
reports['indication'] = reports['indication'].apply(lambda x : x if x != None else '')
reports['findings'] = reports['findings'].apply(lambda x : str(' ') + str(x)) 
reports['text'] = reports['indication'] + reports['findings']

filtered_reports = reports[~reports['text'].isna()]

filtered_reports['text'] = filtered_reports['text'].apply(lambda x : clean_text(x))
filtered_reports['label'] = filtered_reports['Problems'].apply(lambda x : 'normal' if x == 'normal' else 'diseased')
filtered_reports['class'] = filtered_reports['label'].apply(lambda x : 0 if x == 'normal' else 1)

uids = filtered_reports['uid'].unique()
all_images = []
all_text = []
all_labels = []
all_orient = []
all_uids = []


for each in range(len(filtered_reports)): 

    image_df = image_files[image_files['uid'] == filtered_reports.iloc[each, 0]]

       
    for i in range(len(image_df)) :
       
        all_images.append(image_df.iloc[i, 1])
        all_text.append(filtered_reports.iloc[each, 8])
        all_orient.append(image_df.iloc[i,2])
        all_labels.append(filtered_reports.iloc[each, 2])
        all_uids.append(filtered_reports.iloc[each, 0])
        

/var/folders/rp/w7dswjqx7yd_48lc7m55s2wr0000gn/T/ipykernel_1377/1049899772.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reports['text'] = filtered_reports['text'].apply(lambda x : clean_text(x))
/var/folders/rp/w7dswjqx7yd_48lc7m55s2wr0000gn/T/ipykernel_1377/1049899772.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reports['label'] = filtered_reports['Problems'].apply(lambda x : 'normal' if x == 'normal' else 'diseased')
/var/folders/rp/w7dswjqx7yd_48lc7m55s2wr0000gn/T/ipyker

In [ ]:
dataset_df = pd.DataFrame({'images' : all_images, 'text': all_text, 'orient' : all_orient, 'label' : all_labels, 'uid': all_uids})
dataset_df['label'] = dataset_df['label'].apply(lambda x : 'normal' if x == 'normal' else 'diseased')
dataset_df['class'] = dataset_df['label'].apply(lambda x : 0 if x == 'normal' else 1)
#unique_data = dataset.groupby('uid').first()
#unique_data =unique_data.reset_index()

In [ ]:

def infonce_loss(logits, temperature=0.07):
    
    batch_size = logits.size(0)
    labels = torch.arange(batch_size).to(logits.device)
    
    text_loss = F.cross_entropy(logits / temperature, labels)
    
    image_loss = F.cross_entropy(logits.T / temperature, labels)
    
    total_loss = (text_loss + image_loss) / 2.0
    
    return total_loss

class CLIPModel(nn.Module):
    def __init__(self, dropout,temperature=0.07):
        super().__init__()
        
        # self.image_encoder = models.resnet50(weights='IMAGENET1K_V1')
       
        # self.image_encoder.fc = nn.Linear(2048, 512)  # 512 dimensions

        self.image_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224')
        #self.image_projection = nn.Linear(768, 512)  # ViT outputs 768-dim features
        
        
        self.text_encoder = AutoModel.from_pretrained('bert-base-uncased')
        #self.text_projection = nn.Linear(768, 512)  # 512 dimensions
        self.image_projection = nn.Sequential(
            nn.Linear(768, 768),  # Same size first
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(768, 512)
        )
        
        self.text_projection = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(768, 512)
        )
        self.temperature = temperature
        
    def encode_image(self, image):
        # image_features = self.image_encoder(image)
        # return F.normalize(image_features, dim=-1)
        
        image_features = self.image_encoder(image).last_hidden_state[:, 0, :]  # Get [CLS] token
        image_features = self.image_projection(image_features)
        return F.normalize(image_features, dim=-1)
        
    
    def encode_text(self, input_ids, attention_mask):
        text_features = self.text_encoder(input_ids=input_ids, 
                                        attention_mask=attention_mask).last_hidden_state[:, 0, :]
        text_features = self.text_projection(text_features)
        return F.normalize(text_features, dim=-1)
    
    def forward(self, image, input_ids, attention_mask):
        image_features = self.encode_image(image)
        text_features = self.encode_text(input_ids, attention_mask)
        #print(text_features.shape, image_features.shape)
        similarity = torch.matmul(text_features, image_features.t())
        
        return similarity

def train_clip(model, train_loader, optimizer, device, epoch):
    model.train()
    total_loss = 0
    i = 0
    for batch in train_loader:
        if i%10 == 0 :
            print(i)
        i += 1
       
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        
        similarity = model(images, input_ids, attention_mask)
        
        loss = infonce_loss(similarity, temperature=model.temperature)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

In [ ]:
def handle_grayscale(x):
    return x.repeat(3, 1, 1) if x.shape[0] == 1 else x

class TextImageDataset(Dataset):
    def __init__(self, dataset, transform=None, max_length=128):
        self.image_paths = dataset['images']
        self.texts = dataset['text']
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Lambda(handle_grayscale),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = self.transform(read_image('/kaggle/input/chest-xrays-indiana-university/images/images_normalized/' + self.image_paths[idx]).float()/255.0)
        
        text = self.texts[idx]
        tokens = self.tokenizer(text, 
                              padding='max_length',
                              max_length=self.max_length,
                              truncation=True,
                              return_tensors='pt')
        
        return {
            'image': image,
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze()
        }

In [ ]:
train_dataset = torch.load('/kaggle/input/chest-xray/train_dataset.pt')
test_dataset = torch.load('/kaggle/input/chest-xray/test_dataset.pt')
val_dataset = torch.load('/kaggle/input/chest-xray/val_dataset.pt')

In [ ]:

# Split dataset into train and validation and test

# dataset = TextImageDataset(dataset_df)
# train_size = int(0.7 * len(dataset))
# val_size = int((len(dataset) - train_size)/2)
# test_size = len(dataset)-train_size-val_size
# train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
# torch.save(train_dataset, 'train_dataset.pt')
# torch.save(test_dataset, 'test_dataset.pt')
# torch.save(val_dataset, 'val_dataset.pt')
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
### Setup Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = CLIPModel(dropout=0.3, temperature=0.1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.02)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                             mode='min', 
                             factor=0.1, 
                             patience=5,
                             min_lr=1e-7)
train_losses = []
val_losses = []
best_val_loss = float('inf')

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    start_time = time.time()
    
    # Training
    train_loss = train_clip(model, train_dataloader, optimizer, device, epoch)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            images = batch['image'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            similarity = model(images, input_ids, attention_mask)
            loss = infonce_loss(similarity, temperature=model.temperature)
            val_loss += loss.item()
    
    val_loss /= len(val_dataloader)
    val_losses.append(val_loss)
    
    # Step scheduler with validation loss
    scheduler.step(val_loss)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '/kaggle/working/best_clip_model.pt')
    
    end_time = time.time()
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    print(f"Time: {end_time - start_time:.2f}s")

# Save final model
torch.save(model.state_dict(), '/kaggle/working/final_clip_model.pt')

NameError: name 'test_dataloader' is not defined